### Lập trình mạng nơ ron hồi tiếp từ đầu
- Trong phần này, ta lập trình từ đầu mô hình ngôn ngữ được giới thiệu trong chương trước. Mô hình này dựa trên mạng nơ ron hồi tiếp ở cấp độ ký tự được huấn luyện trên tiểu thuyết The Time Machine. Ta bắt đầu với việc đọc tập dữ liệu.

In [14]:
from d2l import torch as d2l
import math
import torch
from torch import nn
from torch.nn import functional as F

batch_size, num_steps = 32, 35
data = d2l.TimeMachine(batch_size, num_steps)

In [15]:
print(data.vocab.token_freqs)

[(' ', 32775), ('e', 17838), ('t', 13515), ('a', 11704), ('i', 10138), ('n', 9917), ('o', 9758), ('s', 8486), ('h', 8257), ('r', 7674), ('d', 6337), ('l', 6146), ('m', 4043), ('u', 3805), ('c', 3424), ('f', 3354), ('w', 3225), ('g', 3075), ('y', 2679), ('p', 2427), ('b', 1897), ('v', 1295), ('k', 1087), ('x', 236), ('z', 144), ('j', 97), ('q', 95)]


In [16]:
data_iter = data.get_dataloader(train = True)


In [ ]:
# In ra một minibatch
for X, y in data_iter:
    print(X, y)
    break

tensor([[20,  0,  2,  ..., 10, 15,  8],
        [ 6,  0, 20,  ...,  9, 16, 14],
        [ 6,  5,  0,  ...,  0, 21, 19],
        ...,
        [ 0, 14,  2,  ...,  0, 13,  2],
        [16, 19,  6,  ...,  2, 20,  0],
        [ 0, 24,  9,  ...,  0,  2, 20]]) tensor([[ 0,  2, 15,  ..., 15,  8,  0],
        [ 0, 20,  0,  ..., 16, 14,  6],
        [ 5,  0, 21,  ..., 21, 19,  2],
        ...,
        [14,  2, 26,  ..., 13,  2, 17],
        [19,  6,  0,  ..., 20,  0, 16],
        [24,  9, 26,  ...,  2, 20,  0]])


In [23]:
vocab = data.vocab
print(vocab.to_tokens(y[0]))
str = ""
for chr in vocab.to_tokens(y[0]):
    str += chr
print(str)

[' ', 'a', 'n', ' ', 'a', 'b', 's', 'o', 'l', 'u', 't', 'e', 'l', 'y', ' ', 'u', 'n', 'a', 'c', 'c', 'o', 'u', 'n', 't', 'a', 'b', 'l', 'e', ' ', 't', 'h', 'i', 'n', 'g', ' ']
 an absolutely unaccountable thing 


In [ ]:
###############
train_iter, vocab = data.get_dataloader(train = True), data.vocab

#### 1. Biểu diễn one hot

In [ ]:
# Biểu diễn one-hot trong pytorch
F.one_hot(input = torch.tensor([0, 1, 2, 3]), num_classes=4)

tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])

Kích thước minibatch mà ta lấy mẫu ở mỗi lần là __(Kích thước batch, bước thời gian)__. Hàm one_hot biển đổi một minibatch như vậy thành một tensor 3 chiều với kích thước chiều cuối cùng bằng bộ từ vựng. Chúng ta thường xuyên chuyển vị đầu vào để có đầu ra với kích thước __(bước thời gian, kích thước batch, kích thước bộ từ vựng)__ phù hợp hơn để đưa vào mô hình chuỗi.

In [27]:
print(X.shape)
print(F.one_hot(input = X.T, num_classes= len(vocab)).shape)

torch.Size([32, 35])
torch.Size([35, 32, 28])


#### 2. Khởi tạo tham số mô hình
- Tiếp theo ta khởi tạo các tham số cho mô hình RNN. Số nút ẩn num_hiddens là tham số có thể điều chỉnh.

In [ ]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    # Hidden layer parameters
    W_xh = torch.rand(size = (num_inputs, num_hiddens), device = device)
    W_hh = torch.rand(size = (num_hiddens, num_hiddens), device = device)
    b_h = torch.zeros(size = (num_hiddens, 1), device = device)

    # Output layer parameters
    W_hq = torch.rand(size = (num_hiddens, num_outputs), device = device)
    b_q = torch.zeros(size = (num_outputs, 1), device = device)

    params = [W_xh, W_hh, b_h, W_hq, b_q]

    for param in params:
        param.requires_grad_(True)